## Part 1 - Exploratory Data Analysis

### 1.0. Libraries and data files
***

In [1]:
!apt-get update
# importing the necessary spark environment and getting jdk
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop2.7'

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,221 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu b

In [2]:
# getting the data
!pip install -q wget
!pip install -q findspark
import wget
data_19 = 'https://raw.githubusercontent.com/Gooner12/FIFA_2019/main/FIFA_19_players.csv'
data_21 = 'https://raw.githubusercontent.com/Gooner12/FIFA_2019/main/FIFA_21_players.csv'
dataset_19 = wget.download(data_19)
dataset_21 = wget.download(data_21)

In [3]:
import findspark
findspark.init()
#findspark.find()
from pyspark.sql import SparkSession

### 1.1 Data Exploration
***

In [4]:
# importing the 'FIFA_19.csv' as a Spark dataframe
spark = SparkSession.builder.appName('FIFA_19').config('spark.sql.caseSensitive', 'true').getOrCreate()
sc = spark.sparkContext
# not including header as data while reading so the header value is made true
df_19 = spark.read.format('csv').options(header='true', inferschema='true').load('FIFA_19_players.csv')
df_19.show()

+------+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+--------+-------+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+
|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|               Club|           Club Logo|value(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill Moves|     Work Rate| Body 

In [5]:
df_21 = spark.read.format('csv').options(header='true', inferschema='true').load('FIFA_21_players.csv')
df_21.show()

+---------+--------------------+-----------------+--------------------+---+----------+---------+---------+-----------+-------------------+--------------------+-----------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+--------------------+---------+------------------+--------------------+-------------+------------------+-----------+----------+--------------------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+---------

Merging the 2019 and 2021 dataframe to get the overall, potential, value in 2021 for every player.

In [6]:
# first renaming the id column in fifa 21 database same as the id column in fifa 19 database for merging later
df_21 = df_21.withColumnRenamed('sofifa_id','ID')
# joining the two dataframes on id and only merging overall, potential and value_eur to the fifa 19 database
df = df_19.join(df_21['ID','overall','potential','value_eur'], on=['ID'], how='left_outer')
df.cache()
df.show()

+------+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+--------+-------+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+-------+---------+---------+
|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|               Club|           Club Logo|value(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill

In [7]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- value(M): string (nullable = true)
 |-- wage(K): integer (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nullable = true)
 |-- Contract 

Dealing Missing Values:

In this stage, we examine the dataset to see the necessary operations required on data to ease our analysis. Based on the examination, we develop separate transformers that can perform data manipulation to output the data in a desired format.

In [8]:
# checking the missing values
from pyspark.sql.functions import count, col, when
from pyspark.sql import functions as F
null_values_count = df.select([count(when(F.isnull(cols), cols)).alias(cols) for cols in df.columns])
null_values_count.show()

+---+----+---+-----+-----------+----+-------+---------+----+---------+--------+-------+-------+--------------+------------------------+---------+-----------+---------+---------+---------+--------+-------------+------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+-------+---------+---------+
| ID|Name|Age|Photo|Nationality|Flag|Overall|Potential|Club|Club Logo|value(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill Moves|Work Rate|Body Type|Real Face|Position|Jersey Number|Joined|Loaned From|Contract Valid Until|Height|Height(CM)|Weight|Weight(KG)|Crossing|Finishing|Hea

In this analysis, the attributes that we are concerned with are player attributes that has impacts on players' gameplay such as 'Crossing', ShortPassing', etc. We are only concerned with forwards, midfielders and defenders not goalkeepers. It is seen that all attributes of interests have 48 missing values which include goalkeeping attributes too. So, we need to see if those missing values for different attributes belong to goalkeepers only before performing imputation or deletion of missing values.

In [9]:
# checking the missing values after removing goalkeepers
df_non_goalkeeper = df.filter(df['Position'] != 'GK')
df_non_goalkeeper.select([count(F.when(F.isnull(cols), cols)).alias(cols) for cols in df.columns]).show()

+---+----+---+-----+-----------+----+-------+---------+----+---------+--------+-------+-------+--------------+------------------------+---------+-----------+---------+---------+---------+--------+-------------+------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+-------+---------+---------+
| ID|Name|Age|Photo|Nationality|Flag|Overall|Potential|Club|Club Logo|value(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill Moves|Work Rate|Body Type|Real Face|Position|Jersey Number|Joined|Loaned From|Contract Valid Until|Height|Height(CM)|Weight|Weight(KG)|Crossing|Finishing|Hea

It is seen that no missing values are present in our many attributes of interests for players belonging to positions other than goalkeepers. Important features in our analysis such as value(M), potential and value_eur, still have missing values, so appropriate operations to deal with those missing values should be performed before we proceed to other analysis.

Handling of missing values in Value_2019(M) column:

In [10]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

#df_view = 'soccer_analysis'
df.createOrReplaceTempView('soccer_analysis')

In [22]:
spark.sql("""
          Select * from soccer_analysis Where value_eur Is Not Null AND `value(M)` Is Null
          """).show()

+------+-------------+---+--------------------+------------+--------------------+-------+---------+---------------+--------------------+--------+-------+-------+--------------+------------------------+---------+-----------+--------------+---------+---------+--------+-------------+-----------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+-------+---------+---------+
|    ID|         Name|Age|               Photo| Nationality|                Flag|Overall|Potential|           Club|           Club Logo|value(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill Moves|     Work

It is seen that although we have missing values for players value in 2019, we can find the value of some of those players in 2021. Based on the value in 2021, we perform imputation of missing values in value column in 2019. To do that, we first compare the values in both years and calculate the percentage variation in values in those two years. Using that variation, we calculate the figures for missing values. 

The necessary operation is performed in a separate python file which we will import later.

In [12]:
spark.sql("""
          Select * from soccer_analysis Where `value(M)` Like '%K'
          """).show()

+------+-------------+---+--------------------+--------------+--------------------+-------+---------+--------------------+--------------------+--------+-------+-------+--------------+------------------------+---------+-----------+--------------+---------+---------+--------+-------------+------------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+-------+---------+---------+
|    ID|         Name|Age|               Photo|   Nationality|                Flag|Overall|Potential|                Club|           Club Logo|value(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill 

All the issues seen above have to be dealt before proceeding to further analysis. To deal with those issues, a separate python script has been developed which we will import later on in this analysis to create dataframes that are free from complications seen above.

Creating Pipelines for Data Preprocessing:

In this stage, we create a pipelines for the regression task. 

Note: the modules used to generate the regression dataframe can be found in the github repository for this project.

Here, we import two python scripts containing transformers that we developed according to our examination in the previous section. The first python script is data_preprocessor.py which contains classes such as ValueManipulator, UnifyValue and ValueImputer. The second script is target_generator, and it contains classes such as ProfitGenerator and GrowthGenerator.

In [13]:
!rm -rf FIFA_2019

In [14]:
# only for google colab
!git clone https://github.com/Gooner12/FIFA_2019.git FIFA_2019
!mv "/content/FIFA_2019/data_preprocessor.py" "/content/"
!mv "/content/FIFA_2019/target_generator.py" "/content/"
!mv "/content/FIFA_2019/encoder.py" "/content/"

Cloning into 'FIFA_2019'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 203 (delta 109), reused 138 (delta 72), pack-reused 0
Receiving objects: 100% (203/203), 4.87 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [15]:
from pyspark.ml import Pipeline
import data_preprocessor, target_generator

# defining the stage that changes the units of players' values in 2021 along with renaming columns based on the year 
value_manipulation = data_preprocessor.ValueManipulator()
# defining the stage that standardises the players' values in 2019 to millions
value_standardisation = data_preprocessor.UnifyValue()
# defining the stage that imputes and removes missing values present in players' values in 2019 and 2021
value_imputation = data_preprocessor.ValueImputer()
# defining the stage that generates labels based on profits in two years for regression
regression_target = target_generator.ProfitGenerator()
# defining the stage that generates labels based on the potential to grow in two years
classification_target = target_generator.GrowthGenerator()

DataFrame for Regression:

In [16]:
# creating a pipeline to generate dataframe for regression
pipeline_regression = Pipeline(stages=[value_manipulation, value_standardisation, value_imputation, regression_target])
# fitting the pipeline model and transforming the data as defined
pipeline_model = pipeline_regression.fit(df)
regression_df = pipeline_model.transform(df)

In [17]:
regression_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall_2019: integer (nullable = true)
 |-- Potential_2019: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value_2019(M): double (nullable = true)
 |-- wage(K): integer (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nullable = true)

In [18]:
regression_df.show(5)

+------+-----------------+---+--------------------+-----------+--------------------+------------+--------------+-------------------+--------------------+-------------+-------+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+------------+--------------+-------------+-----------------+
|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall_2019|Potential_2019|               Club|           Club Logo|Value_2019(M)|wage(K)|S

In [24]:
# checking the missing values in regression dataframe
regression_df.select([count(F.when(F.isnull(cols), cols)).alias(cols) for cols in regression_df.columns]).show()

+---+----+---+-----+-----------+----+------------+--------------+----+---------+-------------+-------+-------+--------------+------------------------+---------+-----------+---------+---------+---------+--------+-------------+------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+------------+--------------+-------------+-----------------+
| ID|Name|Age|Photo|Nationality|Flag|Overall_2019|Potential_2019|Club|Club Logo|Value_2019(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill Moves|Work Rate|Body Type|Real Face|Position|Jersey Number|Joined|Loaned From|Contract Valid Un

It is seen that the column wage has 89 missing values, and this column might be important in predicting the players' price. 

In [28]:
# creating a view
regression_df.createOrReplaceTempView('soccer_analysis2')

# selecting the records containing null values in the wage column
spark.sql("""
          Select * from soccer_analysis2 Where `wage(K)` Is Null
          """).show()

+------+-------------+---+--------------------+------------+--------------------+------------+--------------+----+--------------------+-------------+-------+-------+--------------+------------------------+---------+-----------+--------------+---------+---------+--------+-------------+------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+------------+--------------+-------------+--------------------+
|    ID|         Name|Age|               Photo| Nationality|                Flag|Overall_2019|Potential_2019|Club|           Club Logo|Value_2019(M)|wage(K)|Special|Preferred Foot|International Repu

In [34]:
# finding the maximum value of a player whose wage is not disclosed
spark.sql("""
          Select MAX(`Value_2019(M)`), MIN(`Value_2019(M)`), MAX(Age), Min(Age) from soccer_analysis2 Where `wage(K)` Is Null
          """).show()

+------------------+------------------+--------+--------+
|max(Value_2019(M))|min(Value_2019(M))|max(Age)|min(Age)|
+------------------+------------------+--------+--------+
|            10.132|               0.0|      38|      18|
+------------------+------------------+--------+--------+



It is seen that the players whose wages are not disclosed are low value players ranging between 0 and 10 million. This might occur due to numerous reasons such as some players are just starting their professional football, some are at the end of their career or some are not good enough to negotiate their wage. These all reasons might led the players to accept the price their club has offered which is usually minimum wage. Thus, we impute the missing wage with the minimum wage amount.

In [31]:
# selecting the minimum wage
spark.sql("""
          Select MIN(`wage(K)`) from soccer_analysis2 Where `wage(K)` Is Not Null
          """).show()

+------------+
|min(wage(K))|
+------------+
|           1|
+------------+



In [35]:
# imputing the missing wage values with one
regression_df = regression_df.fillna(1, subset=['wage(K)'])

In [37]:
# checking the missing values in regression dataframe
regression_df.select([count(F.when(F.isnull(cols), cols)).alias(cols) for cols in regression_df.columns]).show()

+---+----+---+-----+-----------+----+------------+--------------+----+---------+-------------+-------+-------+--------------+------------------------+---------+-----------+---------+---------+---------+--------+-------------+------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+------------+--------------+-------------+-----------------+
| ID|Name|Age|Photo|Nationality|Flag|Overall_2019|Potential_2019|Club|Club Logo|Value_2019(M)|wage(K)|Special|Preferred Foot|International Reputation|Weak Foot|Skill Moves|Work Rate|Body Type|Real Face|Position|Jersey Number|Joined|Loaned From|Contract Valid Un

In [69]:
regression_df.select(['Value_2019(M)', 'Value_2021(M)']).show(5)

+-------------+-------------+
|Value_2019(M)|Value_2021(M)|
+-------------+-------------+
|        110.5|         67.5|
|         77.0|         46.0|
|        118.5|         90.0|
|        102.0|         87.0|
|         93.0|         58.0|
+-------------+-------------+
only showing top 5 rows



### 3.1 Data Preparation

We try to predict the value of players in two years using the attributes that might impact players' price tags and values.
***

Function:

Feature Engineering:

In this step, we try to find the features that are most correlated with the target (Gain in Two Years). From this step, we will only pick up those attributes who have at least of 10% correlation with the target whether negative or positive. We use both spearman and pearson correlation to determine this. Any attribute that pass this criterion on any of those two tests will be accepted into the next step.

In [59]:
features_col = [ 'Value_2021(M)', 'Age', 'Value_2019(M)', 'wage(K)', 'International Reputation', 'Height(CM)', 'Weight(KG)', 
                      'Crossing', 'Finishing', 'HeadingAccuracy', 
                      'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                      'FKAccuracy', 'LongPassing', 'BallControl', 
                      'Acceleration', 'SprintSpeed', 'Agility', 
                      'Reactions', 'Balance', 'ShotPower', 'Jumping', 
                      'Stamina', 'Strength', 'LongShots', 'Aggression', 
                      'Interceptions', 'Positioning', 'Vision', 'Penalties', 
                      'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'Overall_2019']

vec_assembler = VectorAssembler(inputCols=features_col, outputCol='features')
df_class = vec_assembler.transform(regression_df).select('features')
df_class.show(5)

+--------------------+
|            features|
+--------------------+
|[67.5,31.0,110.5,...|
|[46.0,33.0,77.0,4...|
|[90.0,26.0,118.5,...|
|[87.0,27.0,102.0,...|
|[58.0,27.0,93.0,3...|
+--------------------+
only showing top 5 rows



In [60]:
from pyspark.ml.stat import Correlation
#r1 = Correlation.corr(df_class, 'features', method='pearson').head()
pearson_corr = Correlation.corr(df_class, 'features', 'pearson').collect()[0][0]
print(str(pearson_corr))
#print('Pearson correlation matrix:\n' + str(r1[0]))

DenseMatrix([[ 1.        , -0.02770739,  0.85782914, ...,  0.08731344,
               0.06946364,  0.56321327],
             [-0.02770739,  1.        ,  0.08332409, ...,  0.20606472,
               0.1774763 ,  0.45704163],
             [ 0.85782914,  0.08332409,  1.        , ...,  0.09897944,
               0.07406407,  0.63777141],
             ...,
             [ 0.08731344,  0.20606472,  0.09897944, ...,  1.        ,
               0.96660681,  0.26505528],
             [ 0.06946364,  0.1774763 ,  0.07406407, ...,  0.96660681,
               1.        ,  0.22502964],
             [ 0.56321327,  0.45704163,  0.63777141, ...,  0.26505528,
               0.22502964,  1.        ]])


In [61]:
# creating a datafame from the dense matrix
import pandas as pd

labels = features_col 
#creating a dataframe from the dense matrix
pearson_df = pd.DataFrame(pearson_corr.toArray(), index = [i for i in labels], columns = [i for i in labels])
pearson_df

,Value_2021(M),Age,Value_2019(M),wage(K),International Reputation,Height(CM),Weight(KG),Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Overall_2019
Value_2021(M),1.000000,-0.027707,0.857829,0.687137,0.506146,0.016467,0.031581,0.286624,0.260572,0.187264,0.418474,0.283331,0.365557,0.304619,0.253860,0.311748,0.443295,0.201222,0.206534,0.197761,0.484908,0.113784,0.305394,0.075775,0.237617,0.078977,0.287624,0.154577,0.111360,0.294273,0.340133,0.224936,0.430534,0.110445,0.087313,0.069464,0.563213
Age,-0.027707,1.000000,0.083324,0.147143,0.257786,0.034791,0.216236,0.264051,0.145055,0.344698,0.307773,0.244909,0.105670,0.259670,0.306941,0.306181,0.252234,-0.205127,-0.197892,-0.014238,0.456704,-0.091420,0.305241,0.187817,0.224108,0.359905,0.272781,0.397279,0.280574,0.189634,0.239894,0.249726,0.469837,0.233899,0.206065,0.177476,0.457042
Value_2019(M),0.857829,0.083324,1.000000,0.857582,0.658800,0.024735,0.063187,0.307121,0.295208,0.238121,0.461729,0.337592,0.382817,0.340615,0.301870,0.354520,0.481344,0.173487,0.174622,0.197519,0.554811,0.107327,0.351352,0.103629,0.258625,0.119220,0.331899,0.197143,0.134055,0.325437,0.380261,0.279781,0.499742,0.129472,0.098979,0.074064,0.637771
wage(K),0.687137,0.147143,0.857582,1.000000,0.669772,0.034371,0.082407,0.286224,0.245937,0.249402,0.414034,0.298813,0.329633,0.305859,0.266180,0.321273,0.426720,0.113060,0.119359,0.151289,0.505131,0.076448,0.320703,0.110943,0.211890,0.131756,0.292895,0.211978,0.156390,0.280841,0.331681,0.257917,0.460434,0.147690,0.124624,0.105531,0.577682
International Reputation,0.506146,0.257786,0.658800,0.669772,1.000000,0.031382,0.094443,0.263194,0.223769,0.244356,0.366966,0.310462,0.278352,0.303722,0.276803,0.300436,0.369862,0.026645,0.024692,0.097346,0.455615,0.047601,0.311654,0.106159,0.119417,0.127001,0.277217,0.210858,0.143661,0.253562,0.312008,0.285490,0.443133,0.134907,0.107710,0.089109,0.503091
Height(CM),0.016467,0.034791,0.024735,0.034371,0.031382,1.000000,0.389303,-0.257750,-0.145824,0.294405,-0.101514,-0.126293,-0.243338,-0.207104,-0.179702,-0.101760,-0.151227,-0.295821,-0.218334,-0.340024,0.007137,-0.431444,-0.050383,0.022022,-0.049343,0.344989,-0.155465,0.137542,0.089526,-0.206554,-0.194755,-0.108688,0.020093,0.105350,0.103304,0.087486,0.060618
Weight(KG),0.031581,0.216236,0.063187,0.082407,0.094443,0.389303,1.000000,-0.253189,-0.127819,0.522550,-0.071146,-0.086292,-0.278062,-0.198119,-0.156038,-0.083972,-0.133045,-0.414224,-0.311858,-0.495199,0.119793,-0.647471,0.052404,0.107841,0.029534,0.727302,-0.100514,0.321246,0.182635,-0.185965,-0.202406,-0.060165,0.119175,0.191196,0.182973,0.156786,0.185425
Crossing,0.286624,0.264051,0.307121,0.286224,0.263194,-0.257750,-0.253189,1.000000,0.413573,-0.135119,0.632644,0.472616,0.712890,0.716034,0.619132,0.593206,0.683383,0.439971,0.393444,0.547783,0.461856,0.435351,0.458524,-0.027594,0.351512,-0.176722,0.548852,0.105875,0.126635,0.585557,0.644715,0.371653,0.470053,0.088071,0.086104,0.079624,0.496591
Finishing,0.260572,0.145055,0.295208,0.245937,0.223769,-0.145824,-0.127819,0.413573,1.000000,0.002354,0.397742,0.814534,0.706552,0.613396,0.536832,0.218261,0.645211,0.373790,0.349441,0.481593,0.349369,0.311220,0.702877,-0.057662,0.119002,-0.125564,0.802591,-0.200801,-0.491597,0.819379,0.655078,0.745105,0.411945,-0.502248,-0.558260,-0.585836,0.372765
HeadingAccuracy,0.187264,0.344698,0.238121,0.249402,0.244356,0.294405,0.522550,-0.135119,0.002354,1.000000,0.133048,0.062462,-0.133505,-0.091835,-0.076077,0.057872,0.070801,-0.336139,-0.239271,-0.351510,0.407164,-0.467881,0.190815,0.357258,0.161807,0.641699,0.030432,0.458635,0.286643,-0.023310,-0.078500,0.109355,0.346528,0.274799,0.257301,0.234819,0.467578


Attributes that fail to satisfy the criterion from the pearson test are height(cm), weight(kg), crossing, dribbling, acceleration, sprint speed, agility, balance, jumping, stamina, strength, interceptions, marking, standing tackle and sliding tackle.

In [62]:
# creating a dataframe based on the spearman correlation
spearman_corr = Correlation.corr(df_class, 'features', 'spearman').collect()[0][0]
spearman_df = pd.DataFrame(spearman_corr.toArray(), index = [i for i in labels], columns = [i for i in labels])
spearman_df

,Value_2021(M),Age,Value_2019(M),wage(K),International Reputation,Height(CM),Weight(KG),Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Overall_2019
Value_2021(M),1.000000,-0.031378,0.852911,0.616682,0.320234,0.018145,0.042882,0.391831,0.336049,0.264642,0.573712,0.339630,0.544638,0.383999,3.050288e-01,0.425783,0.630050,0.283449,0.300918,0.278653,0.594043,0.134347,0.436190,0.115608,0.323502,0.120975,0.400749,0.221337,2.012557e-01,0.410170,0.423714,0.286102,0.537944,0.181707,0.189402,0.161119,0.736222
Age,-0.031378,1.000000,0.231094,0.334480,0.268063,0.055577,0.223385,0.290206,0.156343,0.357558,0.310933,0.252197,0.140607,0.271619,3.038601e-01,0.312917,0.265100,-0.127902,-0.112156,0.028833,0.479882,-0.072572,0.339442,0.232837,0.291573,0.377519,0.303410,0.415708,3.124447e-01,0.239529,0.259726,0.246365,0.475939,0.264212,0.245920,0.219029,0.480980
Value_2019(M),0.852911,0.231094,1.000000,0.772021,0.389602,0.050085,0.118722,0.477807,0.415332,0.384919,0.693919,0.439109,0.616439,0.482930,4.053561e-01,0.537003,0.744082,0.263435,0.283599,0.311696,0.769080,0.119048,0.571445,0.199892,0.432679,0.251933,0.519177,0.355802,2.892386e-01,0.514837,0.535376,0.372171,0.717101,0.254781,0.252667,0.209610,0.936872
wage(K),0.616682,0.334480,0.772021,1.000000,0.401885,0.051890,0.123141,0.422553,0.340516,0.377258,0.577342,0.388469,0.494325,0.424884,3.614060e-01,0.452557,0.613872,0.134672,0.152432,0.216585,0.670949,0.066129,0.497334,0.192885,0.325151,0.253610,0.445290,0.350599,2.789709e-01,0.432336,0.446747,0.347812,0.628725,0.242134,0.247475,0.212508,0.793917
International Reputation,0.320234,0.268063,0.389602,0.401885,1.000000,0.038529,0.099149,0.270247,0.212304,0.234937,0.367075,0.278096,0.284361,0.287528,2.496700e-01,0.308814,0.358218,0.024574,0.024530,0.094433,0.417354,0.049071,0.334086,0.123250,0.124442,0.139122,0.288676,0.233360,1.742434e-01,0.263818,0.300652,0.258393,0.419697,0.157131,0.148569,0.126916,0.439964
Height(CM),0.018145,0.055577,0.050085,0.051890,0.038529,1.000000,0.530650,-0.304192,-0.165098,0.403945,-0.132352,-0.158216,-0.284660,-0.256276,-2.374639e-01,-0.123920,-0.183924,-0.374939,-0.273359,-0.435822,0.007320,-0.570699,-0.053159,0.058853,-0.065388,0.506010,-0.173364,0.186022,1.362339e-01,-0.211042,-0.231375,-0.135294,0.020910,0.156654,0.164599,0.136737,0.067064
Weight(KG),0.042882,0.223385,0.118722,0.123141,0.099149,0.530650,1.000000,-0.231116,-0.111530,0.526319,-0.073164,-0.088467,-0.252942,-0.191950,-1.691041e-01,-0.073704,-0.123586,-0.392698,-0.285472,-0.471030,0.117305,-0.634769,0.063621,0.143869,0.024156,0.721054,-0.080012,0.321264,2.054623e-01,-0.139818,-0.189132,-0.060939,0.115193,0.219120,0.221535,0.190744,0.176246
Crossing,0.391831,0.290206,0.477807,0.422553,0.270247,-0.304192,-0.231116,1.000000,0.382331,-0.121629,0.619648,0.460440,0.697100,0.707600,6.100146e-01,0.570590,0.672943,0.451486,0.409914,0.529829,0.475300,0.413912,0.468827,-0.011435,0.382261,-0.177244,0.534594,0.112616,1.466893e-01,0.551418,0.623248,0.357482,0.474868,0.099990,0.102671,0.107060,0.515703
Finishing,0.336049,0.156343,0.415332,0.340516,0.212304,-0.165098,-0.111530,0.382331,1.000000,0.017743,0.363167,0.813244,0.690492,0.587632,5.235825e-01,0.192678,0.626822,0.378769,0.355991,0.462088,0.349026,0.292872,0.690493,-0.050877,0.117522,-0.116566,0.780261,-0.190788,-4.720628e-01,0.821689,0.613399,0.751012,0.396163,-0.498034,-0.538022,-0.567660,0.374860
HeadingAccuracy,0.264642,0.357558,0.384919,0.377258,0.234937,0.403945,0.526319,-0.121629,0.017743,1.000000,0.122471,0.048592,-0.111208,-0.092524,-8.819589e-02,0.061870,0.072667,-0.321094,-0.220789,-0.342045,0.390993,-0.464468,0.205938,0.380576,0.155508,0.659633,0.051143,0.464795,3.318644e-01,0.029467,-0.072339,0.098704,0.340613,0.324002,0.328199,0.298971,0.463664

Attributes that fail to satisfy the criterion from the spearman test are height(cm), acceleration, sprint speed, agility, balance and sliding tackle.

Based on the above two tests, we remove height(cm), acceleration, sprint speed, agility, balance and sliding tackle from the prediction analysis in further steps.

In [63]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

def steps_processor(features_col, df):

  # creating a feature vector
  vec_assembler = VectorAssembler(inputCols=features_col, outputCol='features')
  df_class = vec_assembler.transform(df).select('features','Value_2021(M)')

  # scaling the feature vector
  standardscaler = StandardScaler().setInputCol('features').setOutputCol('Scaled_features')
  raw_data = standardscaler.fit(df_class).transform(df_class)

  # splitting the dataset
  train, test = raw_data.randomSplit([0.7, 0.3], seed=2020)
  return train, test

In [64]:
features_col = ['Age', 'Value_2019(M)', 'wage(K)', 'International Reputation', 'Weight(KG)', 
                      'Crossing', 'Finishing', 'HeadingAccuracy', 
                      'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                      'FKAccuracy', 'LongPassing', 'BallControl', 
                      'Reactions', 'ShotPower', 'Jumping', 
                      'Stamina', 'Strength', 'LongShots', 'Aggression', 
                      'Interceptions', 'Positioning', 'Vision', 'Penalties', 
                      'Composure', 'Marking', 'StandingTackle', 'Overall_2019']
train, test = steps_processor(features_col, regression_df)

In [65]:
test.show()

+--------------------+-------------+--------------------+
|            features|Value_2021(M)|     Scaled_features|
+--------------------+-------------+--------------------+
|[17.0,1.100000023...|         0.95|[3.74211125958050...|
|[17.0,1.700000047...|        0.925|[3.74211125958050...|
|[17.0,1.799999952...|         45.0|[3.74211125958050...|
|[17.0,3.5,8.0,1.0...|          9.0|[3.74211125958050...|
|[17.0,3.5,25.0,1....|         10.0|[3.74211125958050...|
|[17.0,17.5,66.0,1...|         27.5|[3.74211125958050...|
|[18.0,1.0,2.0,1.0...|          1.6|[3.96223545132053...|
|[18.0,1.100000023...|          3.9|[3.96223545132053...|
|[18.0,1.100000023...|          1.1|[3.96223545132053...|
|[18.0,1.200000047...|         12.5|[3.96223545132053...|
|[18.0,1.200000047...|         0.85|[3.96223545132053...|
|[18.0,1.799999952...|          1.4|[3.96223545132053...|
|[18.0,2.200000047...|          2.0|[3.96223545132053...|
|[18.0,2.200000047...|         10.0|[3.96223545132053...|
|[18.0,2.59999

Next, we compare three baseline models and select the best baseline model for further tuning in the later step.

In [66]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='Scaled_features', labelCol='Value_2021(M)', maxIter=10, regParam=0.3, elasticNetParam=0.8)
model_lr = lr.fit(train)
predict_train_lr = model_lr.transform(train)
predict_test_lr = model_lr.transform(test)
predict_test_lr.select('Value_2021(M)', 'prediction').show(10)

+-------------+------------------+
|Value_2021(M)|        prediction|
+-------------+------------------+
|         0.95|2.2853271612424146|
|        0.925|2.7436997653608843|
|         45.0|2.8636500762615213|
|          9.0| 4.206670232060648|
|         10.0| 4.204508393724913|
|         27.5|14.475995378390369|
|          1.6|2.0361919280430483|
|          3.9|2.1043452831497693|
|          1.1| 2.099590396412987|
|         12.5|2.2652032873815804|
+-------------+------------------+
only showing top 10 rows



In [67]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol='Scaled_features', labelCol='Value_2021(M)', maxIter=10)
model_gbt = gbt.fit(train)
predict_train_gbt = model_gbt.transform(train)
predict_test_gbt = model_gbt.transform(test)
predict_test_gbt.select('Value_2021(M)', 'prediction').show(10)

+-------------+------------------+
|Value_2021(M)|        prediction|
+-------------+------------------+
|         0.95| 2.621418120367815|
|        0.925| 4.065375745443163|
|         45.0| 4.065375745443163|
|          9.0|14.269702888732695|
|         10.0|14.269702888732695|
|         27.5| 66.88421438453453|
|          1.6|1.9575207670881831|
|          3.9| 2.621418120367815|
|          1.1| 2.621418120367815|
|         12.5| 2.621418120367815|
+-------------+------------------+
only showing top 10 rows



In [68]:
from pyspark.ml.regression import IsotonicRegression

ir = IsotonicRegression(featuresCol='Scaled_features', labelCol='Value_2021(M)')
model_ir = ir.fit(train)
predict_train_ir = model_ir.transform(train)
predict_test_ir = model_ir.transform(test)
predict_test_ir.select('Value_2021(M)', 'prediction').show(10)

+-------------+----------+
|Value_2021(M)|prediction|
+-------------+----------+
|         0.95|      0.45|
|        0.925|      0.45|
|         45.0|      0.45|
|          9.0|      0.45|
|         10.0|      0.45|
|         27.5|      0.45|
|          1.6|      0.95|
|          3.9|      0.95|
|          1.1|      0.95|
|         12.5|      0.95|
+-------------+----------+
only showing top 10 rows



In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier

def logistic_regression_model(train, test, weight=None):
  dict_lr = {}
  if weight is None:
    lr = LogisticRegression(labelCol='Target', featuresCol='Scaled_features', maxIter=100)
  elif weight is not None:
    lr = LogisticRegression(labelCol='Target', featuresCol='Scaled_features', weightCol=weight, maxIter=100)
  model_lr = lr.fit(train)
  predict_train_lr = model_lr.transform(train)
  predict_test_lr = model_lr.transform(test)
  dict_lr['Logistic Regression'] = [predict_train_lr, predict_test_lr]
  return dict_lr

def random_forest_model(train, test, weight=None):
  dict_rf = {}
  if weight is None:
    rf = RandomForestClassifier(labelCol='Target', featuresCol='Scaled_features', numTrees=20, maxDepth=5,\
                                            impurity='gini', maxBins=32, seed=2020)
  elif weight is not None:
    rf = RandomForestClassifier(labelCol='Target', featuresCol='Scaled_features', numTrees=20, maxDepth=5,\
                                          weightCol=weight, impurity='gini', maxBins=32, seed=2020)
  model_rf = rf.fit(train)
  predict_train_rf = model_rf.transform(train)
  predict_test_rf = model_rf.transform(test)
  dict_rf['Random Forest'] = [predict_train_rf, predict_test_rf]
  return dict_rf

def decision_tree_model(train, test, weight=None):
  dict_dt = {}
  if weight is None:
    dt = DecisionTreeClassifier(labelCol='Target', featuresCol='Scaled_features', maxDepth=5, maxBins=32, impurity='gini', seed=2020)
  if weight is not None:
    dt = DecisionTreeClassifier(labelCol='Target', featuresCol='Scaled_features', weightCol=weight, maxDepth=5, maxBins=32, impurity='gini', seed=2020)
  model_dt = dt.fit(train)
  predict_train_dt = model_dt.transform(train)
  predict_test_dt = model_dt.transform(test)
  dict_dt['Decision Tree'] = [predict_train_dt, predict_test_dt]
  return dict_dt

In [ ]:
def model_selector(model_name_list, train, test, weight=None):
  dict_models = {}
  if any(x in 'Logistic Regression' for x in model_name_list):
    dict_lr = logistic_regression_model(train, test, weight)
    dict_models['Logistic Regression'] = dict_lr.get('Logistic Regression')

  if any(x in 'Random Forest' for x in model_name_list):
    dict_rf = random_forest_model(train, test, weight)
    dict_models['Random Forest'] = dict_rf.get('Random Forest')

  if any(x in 'Decision Tree' for x in model_name_list):
    dict_dt = decision_tree_model(train, test, weight)
    dict_models['Decision Tree'] = dict_dt['Decision Tree']
  return dict_models

In [ ]:
models = ['Logistic Regression', 'Random Forest', 'Decision Tree']
dict_models = model_selector(models, train, test)

In [ ]:
dict_models

{'Decision Tree': [DataFrame[features: vector, Growth_Level: string, Target: int, Scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double],
  DataFrame[features: vector, Growth_Level: string, Target: int, Scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double]],
 'Logistic Regression': [DataFrame[features: vector, Growth_Level: string, Target: int, Scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double],
  DataFrame[features: vector, Growth_Level: string, Target: int, Scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double]],
 'Random Forest': [DataFrame[features: vector, Growth_Level: string, Target: int, Scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double],
  DataFrame[features: vector, Growth_Level: string, Target: int, Scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double]]}

Evaluating the model using confusion matrix.

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

def calculate_confusion_matrix(df):
  # the label need to be in float datatype otherwise it won't work
  # creating a column that represents the target but the datatype is float
  df_float = df.select(['prediction','Target']).withColumn('label', F.col('Target').cast(FloatType()))

  # selecting only prediction and label columns
  df_float = df_float.select(['prediction','label'])
  # casting to rdd and mapping to a tuple
  metrics = MulticlassMetrics(df_float.rdd.map(tuple))
  confusion_matrix = metrics.confusionMatrix().toArray()
  # changing the confusion matrix array datatype to int as it's datatype is float
  confusion_matrix = confusion_matrix.astype('int32')
  
  return confusion_matrix, metrics

In [ ]:
def calculate_performance_metrics_position(metrics):
  no_details = ()
  low_details = ()
  mid_details = ()
  high_details = ()
 
  # calculating precision, recall and f1-score for different labels
  for lab in 0.0,1.0,2.0,3.0:
    if lab == 0:
      precision_no = round(metrics.precision(lab) * 100, 2)
      recall_no = round(metrics.recall(lab) * 100, 2)
      f1_score_no = round(metrics.fMeasure(lab, beta=1.0) * 100, 2)
      no_details = no_details + (precision_no, recall_no, f1_score_no)

    elif lab == 1:
      precision_low = round(metrics.precision(lab) * 100, 2)
      recall_low = round(metrics.recall(lab) * 100, 2)
      f1_score_low = round(metrics.fMeasure(lab, beta=1.0) * 100, 2)
      low_details = low_details + (precision_low, recall_low, f1_score_low)

    elif lab == 2:
      precision_mid = round(metrics.precision(lab) * 100, 2)
      recall_mid = round(metrics.recall(lab) * 100, 2)
      f1_score_mid = round(metrics.fMeasure(lab, beta=1.0) * 100, 2)
      mid_details = mid_details + (precision_mid, recall_mid, f1_score_mid)

    elif lab == 3:
      precision_high = round(metrics.precision(lab) * 100, 2)
      recall_high = round(metrics.recall(lab) * 100, 2)
      f1_score_high = round(metrics.fMeasure(lab, beta=1.0) * 100, 2)
      high_details = high_details + (precision_high, recall_high, f1_score_high)

  return no_details, low_details, mid_details, high_details

Creating tables or dataframes to compare the performance of different classifiers.

In [ ]:
# defining a function that generates comparison tables
from pyspark.sql.types import StructType, StructField
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window

def comparison_table(pos, classifiers, *args):
  # creating a new dataframe from list of classifiers
  #classifiers = ('Random Forest', 'Logistic Regression', 'Decision Tree')
  metrics_detail_list = []
  df_classifier = spark.createDataFrame([(l,) for l in classifiers], ['Classifier'])

  # stating conditions to select data based on player positions
  if pos == 'No_Growth':
    index = 0
    print('No Growth:')
  elif pos == 'Low_Growth':
    index = 1
    print('Low Growth:')
  elif pos == 'Mid_Growth':
    index = 2
    print('Mid Growth:')
  elif pos == 'High_Growth':
    index = 3
    print('High Growth')

  # populating the data for dataframes containing performance indicators
  for i in args:
    metrics_detail_list += [i[index]]
  
  # defining the schema of the dataframes containing performance metrics
  schema = StructType([
      StructField('Precision', FloatType(), False),
      StructField('Recall', FloatType(), False),
      StructField('F1-score', FloatType(), False)
      ])
  
  # creating a dataframe for performance metrics
  pos_df = spark.createDataFrame(metrics_detail_list, schema)

  # adding index to the dataframes to join later
  pos_df = pos_df.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
  df_classifier = df_classifier.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
  comparison_df = df_classifier.join(pos_df, pos_df.row_index == df_classifier.row_index).drop('row_index')
  
  return comparison_df

In [ ]:
confusion_matrix_lr, metrics_lr = calculate_confusion_matrix(dict_models.get('Logistic Regression')[1])
performance_lr = calculate_performance_metrics_position(metrics_lr)
confusion_matrix_rf, metrics_rf = calculate_confusion_matrix(dict_models.get('Random Forest')[1])
performance_rf = calculate_performance_metrics_position(metrics_rf)
confusion_matrix_dt, metrics_dt = calculate_confusion_matrix(dict_models.get('Decision Tree')[1])
performance_dt = calculate_performance_metrics_position(metrics_dt)

In [ ]:
def show_comparison_table(classifiers, *args):
  comparison_table('No_Growth', classifiers, *[x for x in args]).show()
  comparison_table('Low_Growth', classifiers, *[x for x in args]).show()
  comparison_table('Mid_Growth', classifiers, *[x for x in args]).show() 
  comparison_table('High_Growth', classifiers, *[x for x in args]).show() 

classifiers = ('Random Forest', 'Logistic Regression', 'Decision Tree')
show_comparison_table(classifiers, performance_rf, performance_lr, performance_dt) 

No Growth:
+-------------------+---------+------+--------+
|         Classifier|Precision|Recall|F1-score|
+-------------------+---------+------+--------+
|      Random Forest|     83.0| 100.0|   90.71|
|Logistic Regression|     83.0| 100.0|   90.71|
|      Decision Tree|     83.0| 99.87|   90.66|
+-------------------+---------+------+--------+

Low Growth:
+-------------------+---------+------+--------+
|         Classifier|Precision|Recall|F1-score|
+-------------------+---------+------+--------+
|      Random Forest|      0.0|   0.0|     0.0|
|Logistic Regression|      0.0|   0.0|     0.0|
|      Decision Tree|    16.67|  0.23|    0.45|
+-------------------+---------+------+--------+

Mid Growth:
+-------------------+---------+------+--------+
|         Classifier|Precision|Recall|F1-score|
+-------------------+---------+------+--------+
|      Random Forest|      0.0|   0.0|     0.0|
|Logistic Regression|      0.0|   0.0|     0.0|
|      Decision Tree|      0.0|   0.0|     0.0|
+--

All three models have high recall with Random Forest and Logistic Regression having 100% recall for No_growth category. This means our models was able to classify No_growth players correctly all the time. All three models have a precision of 82.97%. This means our models are correct 82.97% of the time when they predict a player belongs to the No_growth category. However, our models performed extremely poor in classifying other categories (low, mid and high growth players). This is because our dataset is heavily imbalanced towards the No_growth players.


To balance the dataset, we use SMOTE upsampling/ downsampling which we developed for upsampling and downsampling multiple classes. 

In [ ]:
# only for colab
!mv "/content/FIFA_2019/smote_multiclass_sampling.py" "/content/"

In [ ]:
# selecting only scaled features and target to up/ downsample
train = train.select(['Scaled_features', 'Target'])
test = test.select(['Scaled_features', 'Target'])

Down: 80
up 1: 500
up 2: 700
up 3: 2000

In [ ]:
import smote_multiclass_sampling as smt
obj = smt.SMOTEMultiClassBalancer(spark_context=sc, target='Target', minority_class=1, majority_class=0, upsample_percentage=500, downsample_percentage=80)
train = obj.transform(train)

Is there any more minority classes? Type Yes or No. yes
Enter minority class:2
Enter upsample percentage:700
Is there any more minority classes? Type Yes or No. yes
Enter minority class:3
Enter upsample percentage:2000
Is there any more minority classes? Type Yes or No. no


In [ ]:
train.groupby('Target').agg({'Scaled_features':'count'}).show()

+------+----------------------+
|Target|count(Scaled_features)|
+------+----------------------+
|     0|                  7497|
|     1|                  6528|
|     3|                  4767|
|     2|                  5096|
+------+----------------------+



In [ ]:
test.groupby('Target').agg({'Scaled_features':'count'}).show()

+------+----------------------+
|Target|count(Scaled_features)|
+------+----------------------+
|     1|                   441|
|     3|                   105|
|     2|                   269|
|     0|                  3980|
+------+----------------------+



For checking SMOTE effects, we select decision tree for classication.

In [ ]:
# using decision tree for classification
model = ['Decision Tree']
dict_models = model_selector(model, train, test)

# obtaining confusion matrix 
confusion_matrix_dt, metrics_dt = calculate_confusion_matrix(dict_models.get('Decision Tree')[1])
performance_dt = calculate_performance_metrics_position(metrics_dt)

# showing the comparison table
print('Performance for decision tree:')
classifier = ('Decision Tree',)
show_comparison_table(classifier, performance_dt) 

Performance for decision tree:
No Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    86.89| 48.97|   62.64|
+-------------+---------+------+--------+

Low Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|     10.4| 38.55|   16.38|
+-------------+---------+------+--------+

Mid Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|     7.28|  5.58|    6.32|
+-------------+---------+------+--------+

High Growth
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|     7.17| 48.57|    12.5|
+-------------+---------+------+--------+



Checking the performance for training sets:

In [ ]:
# obtaining confusion matrix for three classifiers
confusion_matrix_dt, metrics_dt = calculate_confusion_matrix(dict_models.get('Decision Tree')[0])
performance_dt = calculate_performance_metrics_position(metrics_dt)

In [ ]:
# showing the comparison table
print('Performance for training data:')
classifier = ('Decision Tree',)
show_comparison_table(classifier, performance_dt) 

Performance for training data:
No Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    49.69| 48.86|   49.27|
+-------------+---------+------+--------+

Low Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    40.83| 54.96|   46.85|
+-------------+---------+------+--------+

Mid Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|     45.4| 12.11|   19.12|
+-------------+---------+------+--------+

High Growth
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    47.24| 63.12|   54.04|
+-------------+---------+------+--------+



It seems that using synthetic samples in training process caused our model to have high variance in general as the performance in validation set is very low for all groups except for "No Growth" category compared to the training set performance. Having more data would help in reducing the generalising error and make more accurate prediction on the validation set. Despite this, we try other techniques such as assigning weights to classes and feature engineering to see if we can improve the performance.

Adjusing Weights:

In this part, we create a new column called weight in which priority is assigned based on their presence in the dataset such that high frequency classes are given low value and vice-versa. First, we determine class weights for different classes. Then, map each row containing a label to its class weight. To do that we create a list containing tuples of class and its weight using items method. Then we create a single iterable from a series of iterables generated from unpacking the results from items method using the chain method. Then we create literal columns from the values from the iterable, which is then converted to MapType. Using the new custom map, we map the label to its corresponding weight.

In [ ]:
import numpy as np
from itertools import chain
from pyspark.sql.functions import lit, create_map, col

def weight_allocator(df):
  # getting a list containing the count for each label
  target_count = df.select('Growth_Level').groupby('Growth_Level').count().collect()
  # creating a list of unique labels
  unique_target = [x['Growth_Level'] for x in target_count]
  # getting the total number of samples in the dataset
  total_samples = sum([x['count'] for x in target_count])
  # storing the total number of labels
  label_count = len(target_count)
  # creating a list to store bin count for each label
  bin_count = [x['count'] for x in target_count]

  # generating the class weights based on the class frequency in the data. This is similar to how weights are calculated in scikit-learn
  class_weights = {key: value  for key, value in zip(unique_target, total_samples / (label_count * np.array(bin_count)))}
  print(class_weights)

  custom_map = create_map([lit(x) for x in chain(*class_weights.items())])
  df = df.withColumn('Weight', custom_map.getItem(col('Growth_Level')))
  return df

In [ ]:
# getting the training and testing dataset containing weights
train, test = steps_processor(features_col, weight_allocator(classification_df), 'Weight')

{'High_Growth': 12.140060240963855, 'Mid_Growth': 4.448675496688741, 'No_Growth': 0.3017970797454137, 'Low_Growth': 2.6360366252452585}


In [ ]:
train.show(5)

+--------------------+------------+------+------------------+--------------------+
|            features|Growth_Level|Target|            Weight|     Scaled_features|
+--------------------+------------+------+------------------+--------------------+
|[16.0,1.0,185.928...|   No_Growth|     0|0.3017970797454137|[3.51872175056621...|
|[17.0,1.0,155.448...|  Mid_Growth|     2| 4.448675496688741|[3.73864185997660...|
|[17.0,1.0,155.448...|  Mid_Growth|     2| 4.448675496688741|[3.73864185997660...|
|[17.0,1.0,155.448...|   No_Growth|     0|0.3017970797454137|[3.73864185997660...|
|[17.0,1.0,155.448...|   No_Growth|     0|0.3017970797454137|[3.73864185997660...|
+--------------------+------------+------+------------------+--------------------+
only showing top 5 rows



In [ ]:
# using decision tree for classification
model = ['Decision Tree']
dict_models = model_selector(model, train, test, 'Weight')


# obtaining confusion matrix 
confusion_matrix_dt, metrics_dt = calculate_confusion_matrix(dict_models.get('Decision Tree')[1])
performance_dt = calculate_performance_metrics_position(metrics_dt)

# showing the comparison table
print('Performance for decision tree:')
classifier = ('Decision Tree',)
show_comparison_table(classifier, performance_dt) 

Performance for decision tree:
No Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    88.12|  49.6|   63.47|
+-------------+---------+------+--------+

Low Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    10.87| 15.87|    12.9|
+-------------+---------+------+--------+

Mid Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|      7.7| 27.88|   12.07|
+-------------+---------+------+--------+

High Growth
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|     5.44| 48.57|    9.79|
+-------------+---------+------+--------+



Classifying samples based on their importance was able to improve the performance compared to the original one, but the overall performance is low compared to the SMOTE performance.

Finding relevant features:

In [ ]:
target_encode = classification_df.withColumn('Target', when(col('Growth_Level') == 'No_Growth', 0).when(col('Growth_Level') == 'Low_Growth', 1) \
                               .when(col('Growth_Level') == 'Mid_Growth', 2).otherwise(3))
target_encode.show()

+------+-----------------+---+--------------------+-----------+--------------------+------------+--------------+-------------------+--------------------+-------------+-------+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+----------+------+----------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+-----------------+------------+--------------+-------------+------------+------+
|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall_2019|Potential_2019|               Club|           Club Logo|Value_2019(M)|wage(K)

In [ ]:
features_col = ['Target','Age', 'International Reputation', 'Height(CM)', 'Weight(KG)', 
                      'Crossing', 'Finishing', 'HeadingAccuracy', 
                      'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                      'FKAccuracy', 'LongPassing', 'BallControl', 
                      'Acceleration', 'SprintSpeed', 'Agility', 
                      'Reactions', 'Balance', 'ShotPower', 'Jumping', 
                      'Stamina', 'Strength', 'LongShots', 'Aggression', 
                      'Interceptions', 'Positioning', 'Vision', 'Penalties', 
                      'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'Overall_2019']

vec_assembler = VectorAssembler(inputCols=features_col, outputCol='features')
df_class = vec_assembler.transform(target_encode).select('features')
df_class.show(3)

+--------------------+
|            features|
+--------------------+
|[0.0,31.0,5.0,173...|
|[0.0,33.0,5.0,188...|
|[0.0,26.0,5.0,179...|
+--------------------+
only showing top 3 rows



In [ ]:
from pyspark.ml.stat import Correlation
#r1 = Correlation.corr(df_class, 'features', method='pearson').head()
pearson_corr = Correlation.corr(df_class, 'features', 'pearson').collect()[0][0]
print(str(pearson_corr))
#print('Pearson correlation matrix:\n' + str(r1[0]))

DenseMatrix([[ 1.00000000e+00, -1.75710526e-01, -6.08602475e-02, ...,
               4.07615868e-04,  1.06768080e-02, -1.41511201e-01],
             [-1.75710526e-01,  1.00000000e+00,  2.57815242e-01, ...,
               2.05381502e-01,  1.77006568e-01,  4.55509247e-01],
             [-6.08602475e-02,  2.57815242e-01,  1.00000000e+00, ...,
               1.06882509e-01,  8.82955673e-02,  5.02080103e-01],
             ...,
             [ 4.07615868e-04,  2.05381502e-01,  1.06882509e-01, ...,
               1.00000000e+00,  9.66533074e-01,  2.64081857e-01],
             [ 1.06768080e-02,  1.77006568e-01,  8.82955673e-02, ...,
               9.66533074e-01,  1.00000000e+00,  2.24364834e-01],
             [-1.41511201e-01,  4.55509247e-01,  5.02080103e-01, ...,
               2.64081857e-01,  2.24364834e-01,  1.00000000e+00]])


In [ ]:
# creating a datafame from the dense matrix
import pandas as pd

labels = features_col 
#creating a dataframe from the dense matrix
pearson_df = pd.DataFrame(pearson_corr.toArray(), index = [i for i in labels], columns = [i for i in labels])
pearson_df

,Target,Age,International Reputation,Height(CM),Weight(KG),Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Overall_2019
Target,1.000000,-0.175711,-0.060860,-0.019941,-0.051128,-0.046436,-0.076593,-0.077617,-0.085109,-0.086936,-0.053248,-0.072599,-0.077852,-0.082091,-0.079243,0.023498,0.027971,-0.027709,-0.117439,-0.005130,-0.102695,-0.042050,-0.038485,-0.076072,-0.095342,-0.070870,-0.022285,-0.065162,-0.081502,-0.085040,-0.132055,-0.024756,0.000408,0.010677,-0.141511
Age,-0.175711,1.000000,0.257815,0.035493,0.216992,0.263204,0.144203,0.344275,0.306159,0.244419,0.103602,0.258692,0.305848,0.304539,0.250561,-0.206653,-0.199765,-0.016755,0.454947,-0.092422,0.303636,0.185675,0.220672,0.358897,0.271537,0.396263,0.279857,0.187764,0.238707,0.248733,0.468578,0.232917,0.205382,0.177007,0.455509
International Reputation,-0.060860,0.257815,1.000000,0.030486,0.094125,0.263117,0.223799,0.243389,0.366256,0.310106,0.277794,0.303458,0.276931,0.300089,0.369094,0.025968,0.024052,0.096901,0.454234,0.047210,0.311419,0.105714,0.118857,0.126454,0.276826,0.210173,0.143063,0.253360,0.312202,0.285083,0.441995,0.134482,0.106883,0.088296,0.502080
Height(CM),-0.019941,0.035493,0.030486,1.000000,0.389144,-0.256877,-0.145518,0.294367,-0.101685,-0.125823,-0.243304,-0.206714,-0.179354,-0.101614,-0.151029,-0.295382,-0.217696,-0.339714,0.006939,-0.431352,-0.050375,0.022189,-0.048701,0.344866,-0.154964,0.138249,0.089665,-0.206264,-0.194460,-0.109432,0.019474,0.105447,0.103922,0.088246,0.060495
Weight(KG),-0.051128,0.216992,0.094125,0.389144,1.000000,-0.253124,-0.127715,0.523032,-0.071518,-0.086085,-0.278601,-0.197374,-0.155941,-0.083916,-0.133126,-0.414036,-0.311849,-0.495194,0.119991,-0.647838,0.051881,0.108816,0.028837,0.727493,-0.100292,0.322663,0.183425,-0.186056,-0.202134,-0.060476,0.119253,0.191365,0.183352,0.157271,0.185436
Crossing,-0.046436,0.263204,0.263117,-0.256877,-0.253124,1.000000,0.413404,-0.134959,0.632386,0.472485,0.712261,0.714762,0.618580,0.592781,0.682796,0.439799,0.393403,0.547590,0.461861,0.435751,0.457531,-0.027529,0.351862,-0.177255,0.548462,0.105539,0.125985,0.585624,0.643948,0.371606,0.469666,0.086971,0.085429,0.078963,0.496603
Finishing,-0.076593,0.144203,0.223799,-0.145518,-0.127715,0.413404,1.000000,0.002099,0.398895,0.814663,0.706635,0.614189,0.537508,0.218965,0.645849,0.373630,0.349087,0.480792,0.349146,0.311487,0.703408,-0.057519,0.119415,-0.125550,0.803036,-0.201407,-0.491733,0.818964,0.655283,0.744835,0.412177,-0.503042,-0.558565,-0.585841,0.373079
HeadingAccuracy,-0.077617,0.344275,0.243389,0.294367,0.523032,-0.134959,0.002099,1.000000,0.131909,0.062357,-0.134535,-0.091652,-0.076517,0.057267,0.069890,-0.335712,-0.238734,-0.351398,0.405671,-0.467826,0.189628,0.357951,0.161594,0.641662,0.029784,0.458631,0.286880,-0.024105,-0.079291,0.108405,0.344770,0.274280,0.257420,0.234933,0.466908
ShortPassing,-0.085109,0.306159,0.366256,-0.101685,-0.071518,0.632386,0.398895,0.131909,1.000000,0.469096,0.638983,0.614810,0.584412,0.842294,0.800317,0.171664,0.140618,0.361314,0.658269,0.252552,0.556794,0.026758,0.375038,0.041784,0.575126,0.305331,0.287053,0.501792,0.724511,0.384587,0.673243,0.244696,0.235083,0.203009,0.722720
Volleys,-0.086936,0.244419,0.310106,-0.125823,-0.086085,0.472485,0.814663,0.062357,0.469096,1.000000,0.674461,0.691624,0.618031,0.313951,0.657246,0.311606,0.282198,0.447630,0.433796,0.294044,0.732058,-0.019984,0.146353,-0.074828,0.786780,-0.064275,-0.334536,0.745740,0.659091,0.730061,0.504112,-0.357150,-0.402222,-0.430056,0.452090


From the above, balance and standing tackle have the lowest correlations with the target. Other attributes have somewhat similar level of correlation with the target with age, composure, shot power, reaction and overall having more correlation than all.

In [ ]:
# creating a dataframe based on the spearman correlation
spearman_corr = Correlation.corr(df_class, 'features', 'spearman').collect()[0][0]
spearman_df = pd.DataFrame(spearman_corr.toArray(), index = [i for i in labels], columns = [i for i in labels])
spearman_df

,Target,Age,International Reputation,Height(CM),Weight(KG),Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Overall_2019
Target,1.000000,-0.154658,-0.061528,-0.023833,-0.049550,-0.021515,-0.067753,-0.066174,-0.058526,-0.073823,-0.035489,-0.052840,-0.058291,-0.058951,-0.050663,0.029846,0.033681,-0.017929,-0.089103,0.003263,-0.084378,-0.035705,-0.016091,-0.067118,-0.080031,-0.052384,-0.015368,-0.060700,-0.067029,-0.076075,-0.103894,-0.017234,0.002370,0.013130,-0.104880
Age,-0.154658,1.000000,0.268122,0.056405,0.223873,0.289607,0.155768,0.357441,0.309476,0.251957,0.139012,0.270767,0.303062,0.311688,0.263914,-0.128334,-0.112715,0.027147,0.478780,-0.073338,0.338441,0.231442,0.289909,0.376666,0.302594,0.414853,0.311701,0.238161,0.258643,0.245713,0.475173,0.263264,0.245086,0.218444,0.479806
International Reputation,-0.061528,0.268122,1.000000,0.037842,0.099006,0.270094,0.212450,0.234321,0.366572,0.278036,0.283724,0.287159,0.249683,0.308572,0.357699,0.024138,0.023911,0.093870,0.416324,0.048414,0.333934,0.123058,0.124211,0.138459,0.288289,0.232642,0.173542,0.263858,0.300996,0.257911,0.419054,0.156729,0.147856,0.126175,0.439018
Height(CM),-0.023833,0.056405,0.037842,1.000000,0.530877,-0.303453,-0.165074,0.404135,-0.132477,-0.158019,-0.284976,-0.255939,-0.237088,-0.123891,-0.183863,-0.375159,-0.273081,-0.435693,0.007125,-0.570870,-0.053292,0.059493,-0.064879,0.506191,-0.173205,0.187309,0.136609,-0.211148,-0.231128,-0.135934,0.020635,0.156830,0.165349,0.137720,0.066955
Weight(KG),-0.049550,0.223873,0.099006,0.530877,1.000000,-0.230857,-0.111433,0.526845,-0.073312,-0.088207,-0.253185,-0.191185,-0.168860,-0.073513,-0.123465,-0.392714,-0.285456,-0.470940,0.117846,-0.635116,0.063292,0.145069,0.023822,0.721103,-0.079712,0.322751,0.206314,-0.139943,-0.188731,-0.061041,0.115484,0.219309,0.221949,0.191295,0.176510
Crossing,-0.021515,0.289607,0.270094,-0.303453,-0.230857,1.000000,0.382308,-0.121252,0.619412,0.460497,0.696292,0.706337,0.609316,0.569968,0.672509,0.451645,0.410193,0.529601,0.475246,0.414245,0.468409,-0.011345,0.382866,-0.177685,0.534362,0.112427,0.145873,0.551624,0.622511,0.357770,0.474659,0.099143,0.102113,0.106476,0.515877
Finishing,-0.067753,0.155768,0.212450,-0.165074,-0.111433,0.382308,1.000000,0.017527,0.364291,0.813460,0.690825,0.588595,0.524407,0.193248,0.627501,0.378796,0.355730,0.461267,0.348767,0.293251,0.691131,-0.050757,0.117687,-0.116742,0.780785,-0.191371,-0.471972,0.821560,0.614040,0.750804,0.396771,-0.498686,-0.538139,-0.567594,0.375347
HeadingAccuracy,-0.066174,0.357441,0.234321,0.404135,0.526845,-0.121252,0.017527,1.000000,0.121458,0.048690,-0.112185,-0.092235,-0.088430,0.061291,0.071873,-0.320988,-0.220474,-0.342127,0.389717,-0.464383,0.204966,0.381280,0.155186,0.659558,0.050609,0.464983,0.332025,0.028503,-0.072796,0.098030,0.339576,0.323605,0.328275,0.299059,0.463128
ShortPassing,-0.058526,0.309476,0.366572,-0.132477,-0.073312,0.619412,0.364291,0.121458,1.000000,0.452319,0.634333,0.607459,0.591133,0.843581,0.785602,0.159439,0.127481,0.337707,0.662325,0.252234,0.544236,0.036041,0.400341,0.032367,0.576573,0.314092,0.334867,0.499822,0.739955,0.371738,0.667965,0.277133,0.282220,0.243944,0.720882
Volleys,-0.073823,0.251957,0.278036,-0.158019,-0.088207,0.460497,0.813460,0.048690,0.452319,1.000000,0.682205,0.684523,0.619787,0.309181,0.656412,0.324888,0.295431,0.449631,0.422483,0.294868,0.733877,-0.021083,0.157516,-0.082279,0.784413,-0.059167,-0.310799,0.754059,0.645156,0.730060,0.485656,-0.345976,-0.378999,-0.403994,0.435362


From the above age, overall and composure have higher correlation with the target. Balance and standing tackle have lowest correlation with the target.

Predicting using decision tree with the new feature set along with applying weights:

In [ ]:
# using only the relevant features
features_col = ['Age', 'International Reputation', 'Height(CM)', 'Weight(KG)', 
                      'Crossing', 'Finishing', 'HeadingAccuracy', 
                      'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                      'FKAccuracy', 'LongPassing', 'BallControl', 
                      'Acceleration', 'SprintSpeed', 'Agility', 
                      'Reactions', 'ShotPower', 'Jumping', 
                      'Stamina', 'Strength', 'LongShots', 'Aggression', 
                      'Interceptions', 'Positioning', 'Vision', 'Penalties', 
                      'Composure', 'Marking', 'SlidingTackle', 'Overall_2019']

train, test = steps_processor(features_col, weight_allocator(classification_df), 'Weight')

{'High_Growth': 12.140060240963855, 'Mid_Growth': 4.448675496688741, 'No_Growth': 0.3017970797454137, 'Low_Growth': 2.6360366252452585}


In [ ]:
# using decision tree for classification
model = ['Decision Tree']
dict_models = model_selector(model, train, test, 'Weight')

# obtaining confusion matrix 
confusion_matrix_dt, metrics_dt = calculate_confusion_matrix(dict_models.get('Decision Tree')[1])
performance_dt = calculate_performance_metrics_position(metrics_dt)

# showing the comparison table
print('Performance for decision tree:')
classifier = ('Decision Tree',)
show_comparison_table(classifier, performance_dt) 

Performance for decision tree:
No Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    88.12|  49.6|   63.47|
+-------------+---------+------+--------+

Low Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|    10.87| 15.87|    12.9|
+-------------+---------+------+--------+

Mid Growth:
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|      7.7| 27.88|   12.07|
+-------------+---------+------+--------+

High Growth
+-------------+---------+------+--------+
|   Classifier|Precision|Recall|F1-score|
+-------------+---------+------+--------+
|Decision Tree|     5.44| 48.57|    9.79|
+-------------+---------+------+--------+



It is seen, after removing less correlated features and adjusting weights, the performance remained the same as the performance obtained after only adjusting weights.